# Übung 2

## Aufgabe 1

**Erläutern Sie diese Begriffe anhand des folgenden Beispiels: Gegeben sei eine Relation `Händler(Id, Name, Adresse, Telefonnummer, Branche, URL)`, in der Informationen für ein Händlerverzeichnis gespeichert werden.**

- Welche Attribute oder Attributkombinationen sind Schlüsselkandidaten? 
- Welcher der Schlüsselkandidaten bietet sich als Primärschlüssel an?

Begründen Sie Ihre Wahl und machen Sie Ihre Annahmen über die Bedeutung und möglichen Ausprägungen der Attribute explizit

## Aufgabe 1 - Lösung

**Mögliche Schlüsselkandidaten:**
- `Id`
- `Telefonnummer`

Schlüsselkandidaten sind Attribute oder Attributkombinationen, die eindeutig sind und zur Identifikation eines Datensatzes dienen. In diesem Fall sind `Id` und `Telefonnummer` Schlüsselkandidaten, da sie eindeutig sind und zur Identifikation eines Händlers dienen können.

**Primärschlüssel:**
- `Id`

Der Primärschlüssel ist einzigartig und wird vom System automatisch beim anlegen erstellt.

## Aufgabe 2

**Geben Sie ein SQL-Script an, das die folgenden Tabellen anlegt:**

- `person(nr, name, vorname, plz, strasse)`
- `plzort(plz, ort)`

Überlegen Sie sich geeignete Datentypen und legen Sie dabei folgende Integrity Constraints an:

- das Feld ort in der Tabelle plzort darf nicht leer sein;
- der Wert plz in der Tabelle person muss in der Tabelle plzort vorkommen;
- wenn in plzort eine plz geändert wird, soll diese Änderung bei allen betroffenen personen automatisch nachgezogen werden;
- das Löschen eines Datensatzes aus plzort soll unterbunden werden, wenn in diesem ort noch personen wohnen;
- die Kombination vorname, name soll eindeutig sein;
- die plz darf nur aus Ziffern bestehen (Hinweis: SIMILAR TO1);

## Aufgabe 2 - Lösung

```sql
CREATE TABLE plzort (
    plz VARCHAR(10) PRIMARY KEY,
    ort VARCHAR(100) NOT NULL,
    CHECK (plz SIMILAR TO '[0-9]+')
);

CREATE TABLE person (
    nr SERIAL PRIMARY KEY,
    name VARCHAR(100),
    vorname VARCHAR(100),
    plz VARCHAR(10),
    strasse VARCHAR(100),
    UNIQUE (vorname, name),
    FOREIGN KEY (plz) REFERENCES plzort(plz)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);
```

## Aufgabe 3

Nehmen Sie an, dass die oben definierten Tabellen die folgenden Daten enthalten:

**plzort:**
| plz | ort |
|-----|-----|
| 47805 | Krefeld |

**person:**
| nr | name | vorname | plz | strasse |
|----|------|---------|-----|---------|
| 4711 | Gans | Gustav | 47805 | Kölner Straße 66 |

**Geben Sie Beispiele für INSERT, UPDATE und DELETE-Befehle an, die aufgrund der in Aufgabe 2.2 definierten Constraints fehlschlagen. Welche Statements schlagen warum fehl?**

## Aufgabe 3 - Lösung

**INSERT:**

Einfügen eines Datensatzes in `plzort` mit leerem Ort (Verstoß gegen NOT-NULL Constraint):
```sql
INSERT INTO plzort (plz, ort) VALUES ('47805', '');
```

Einfügen eines Datensatzes in `person` mit einem Wert in `plz`, der nicht in `plzort` referenziert wird (Verstoß gegen FOREIGN KEY Constraint):
```sql
INSERT INTO person (name, vorname, plz, strasse) VALUES ('Müller', 'Peter', '47806', 'Hauptstraße 1');
```

**UPDATE:**

Ändern der PLZ in `plzort` auf einen Wert, der in `person` referenziert wird (Verstoß gegen FOREIGN KEY Constraint):
```sql
UPDATE plzort SET plz = '47806' WHERE plz = '47805';
```

**DELETE:**

Löschen eines Datensatzes in `plzort`, der in `person` referenziert wird (Verstoß gegen RESTRICT Constraint):
```sql
DELETE FROM plzort WHERE plz = '47805';
```


## Aufgabe 4

Mit dem ALTER TABLE Befehl können Spalten zu einer Tabelle hinzugefügt oder entfernt werden:

- `ALTER TABLE table ADD COLUMN column type`
- `ALTER TABLE table DROP COLUMN column`
- `ALTER TABLE table RENAME COLUMN oldname TO newname`

Der Datentyp einer Spalte lässt sich in PostgreSQL wie folgt ändern:
- `ALTER TABLE table ALTER COLUMN column TYPE newtype [USING expression]`

Die optionale Klausel `USING expression` gibt dabei einen Ausdruck an, der die Werte
vom alten Spaltentyp in den neuen Spaltentyp umwandelt. Wenn die `USING`-Klausel fehlt, versucht PostgreSQL, die alten Werte über implizites Casting in den neuen Datentyp zu wandeln.

a) **Nehmen Sie an, dass Sie person.plz zunächst ungünstigerweise als INTEGER
definiert hatten. Wie lautet der Befehl, um in PostgreSQL die Spalte person.plz
von INTEGER in VARCHAR(5) zu ändern?**

b) **In manchen DBMS existiert kein eigener Befehl zum Ändern des Datentyps einer
Spalte. Die Änderung eines Spaltentyps lässt sich dann nur über einen Umweg
erreichen. Überlegen Sie, welcher Umweg das ist und geben Sie eine Folge von
SQL-Statements für das obige Beispiel an.**

## Aufgabe 4 - Lösung

a) 
```sql
ALTER TABLE person ALTER COLUMN plz TYPE VARCHAR(5);
```

b) 
```sql
ALTER TABLE person ADD COLUMN plz_new VARCHAR(5);
UPDATE person SET plz_new = plz;
ALTER TABLE person DROP COLUMN plz;
ALTER TABLE person RENAME COLUMN plz_new TO plz;
```

## Aufgabe 5

** Geben Sie für folgende Datenmanipulationen an den obigen Tabellen SQL-Statements an:**

a) Auflisten aller nr in person, bei denen mindestens eines der restlichen Felder leer (NULL) ist;
b) Anhängen des Strings ’bla’ an jeden namen in person (Hinweis: der Operator für String-Konkatenation lautet in SQL ||);
c) Auflisten der Tupel in person, bei denen die strasse keine Hausnummer hat (Hinweis: regex mit SIMILAR TO);
d) Auflisten aller namen und vornamen von personen zusammen mit dem ort, in dem sie wohnen;

## Aufgabe 5 - Lösung

a) 
```sql
SELECT nr FROM person WHERE name IS NULL OR vorname IS NULL OR plz IS NULL OR strasse IS NULL;
```

b) 
```sql
UPDATE person SET name = name || 'bla';
```

c) 
```sql
SELECT * FROM person WHERE strasse SIMILAR TO '%[^0-9]%';
```

d) 
```sql
SELECT p.name, p.vorname, po.ort
FROM person p
JOIN plzort po ON p.plz = po.plz;
```


